In [1]:
import os
import pandas as pd
import numpy as np
import re
import datetime
import pickle

os.chdir('..')
print(os.getcwd())
from utils import get_fulltext
os.chdir('./data_processing/')

/Users/yiweiluo/scientific-debates


In [2]:
df = pd.read_pickle('../data_scraping/dedup_combined_df.pkl')
df.head(10)

,url,title,date,domain,stance,topic,is_AP,year,pretty_domain
0,www.buzzfeednews.com/article/tasneemnashrulla/...,"""eat the babies"" viral video at aoc town hall ...",2019-10-04 00:00:00,buzzfeed,pro,cc,None,2019.0,Buzzfeed
1,www.buzzfeednews.com/article/passantino/extrem...,"""extremely likely"" global warming is man-made,...",2013-09-27 00:00:00,buzzfeed,pro,cc,None,2013.0,Buzzfeed
2,shotofprevention.com/2010/11/03/history-makes-...,"""history"" makes headlines with launch of new w...",2020-03-13 14:32:02,https://shotofprevention/,pro,vax,False,2020.0,Shot of Prevention
3,www.buzzfeednews.com/article/andrewkaczynski/i...,"""it's global warming, stupid"" - buzzfeed news",2012-11-01 00:00:00,buzzfeed,pro,cc,None,2012.0,Buzzfeed
4,www.buzzfeednews.com/article/tasneemnashrulla/...,"""japan dropped an atomic bomb on america durin...",2014-02-24 00:00:00,buzzfeed,pro,cc,None,2014.0,Buzzfeed
5,www.buzzfeednews.com/article/llevin/opinion-im...,"""look at my record, child"": joe biden showed m...",2019-10-31 00:00:00,buzzfeed,pro,cc,None,2019.0,Buzzfeed
6,www.breitbart.com/politics/2019/06/14/orourke-...,"""president o'rourke will end oil and gas lease...",2019-06-14 00:00:00,breitbart,anti,cc,None,2019.0,Breitbart
7,www.buzzfeednews.com/article/andrewkaczynski/s...,smoking doesnt kill and other great old opeds ...,2015-03-31 00:00:00,buzzfeed,pro,cc,None,2015.0,Buzzfeed
8,www.foxnews.com/world/100-carbon-tax-by-2030-c...,"$100 carbon tax by 2030 could save climate, sa...",2017-05-29 00:00:00,fox,anti,cc,None,2017.0,Fox
21585,childrenshealthdefense.org/news/4-billion-and-...,$4 billion and growing: u.s. payouts for vacc...,2018-11-19 00:00:00,chd,anti,vax,None,2018.0,Children's Health Defense


In [3]:
get_fulltext(df.url.values[0])

['The president and the far right slammed Rep. Alexandria Ocasio-Cortez over the bizarre town hall speaker who suggested eating babies to solve the climate crisis. But the pro-Trump LaRouche PAC, which believes climate change is a hoax, said it planted the troll. President Donald Trump, his son Don Jr., and far-right conservatives attacked Democratic Rep. Alexandria Ocasio-Cortez after a speaker at her town hall meeting Thursday night went on a bizarre rant about how eating babies is the only solution to the climate crisis. But it turns out the stunt was staged by what is now a far-right pro-Trump conspiracy group that compares climate change activism to “genocide.” Trump called Ocasio-Cortez a “wack job” on Twitter after his allies widely shared the video of the unidentified woman saying that Ocasio-Cortez’s Green New Deal was not enough to solve the climate crisis. AOC is a Wack Job! https://t.co/LU3hIeek0c  “We’re not going to be here for much long because of the climate crisis,” th

# Contents:
* [QuoteAnnotator](#QuoteAnnotator)
    * [Climate change](#Climate-change)
        * [URLs from search](#URLs-from-search)
        * [MediaCloud URLs](#MediaCloud-urls)
        * [Reid Google search URLs](#Reid-Urls)
    * [Vaccines](#Vaccines)

# QuoteAnnotator

In [11]:
from pycorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('http://localhost:9000')

'/Users/yiweiluo/scientific-debates/data_processing'